<a href="https://colab.research.google.com/github/Liying1996/DL_Drugs/blob/main/Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import median_absolute_deviation

from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestRegressorrom sklearn.svm import SVR 
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

from scipy.stats import pearsonr

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# datatloader
class RegressionDataset(Dataset):   
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


# AE framework

class AE(nn.Module):
    def __init__(self, input_size):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 1000),
            nn.LeakyReLU(),
            nn.Linear(1000, 500),
            nn.LeakyReLU(),
            nn.Linear(500, 100),
            nn.LeakyReLU(),

        )
        self.decoder = nn.Sequential(
            nn.Linear(100, 500),
            nn.LeakyReLU(),
            nn.Linear(500, 1000),
            nn.LeakyReLU(),
            nn.Linear(1000, input_size),
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoder = self.encoder(x)
        decoder = self.decoder(encoder)
        return encoder,decoder


In [ ]:
# 将所有的步骤封装为pipeline

# 选取top variable genes
def top_mad(X_train, X_test, cut):    
    index = np.argsort(-median_absolute_deviation(X_train))
    new_index = index[:int(X_train.shape[1] * cut)]
    X_train = X_train[:,new_index]
    X_test = X_test[:,new_index]
    return X_train, X_test


# Min-Max标准化

def scale_data(X_train, X_test):
    scaler = MinMaxScaler()
    new_X_train = scaler.fit_transform(X_train)
    new_X_test = scaler.transform(X_test)
    return new_X_train, new_X_test



# AE train
def AE_train(input_size, train_loader, test_loader):
    model = AE(input_size).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  
    # all_train_loss = []
    # all_val_loss = []

    for epoch in range(50):
        train_loss = 0
        # model.train()
        for batchidx, (x, label) in enumerate(train_loader):   
            model.train()   
            x = x.cuda()
            _, x_hat = model(x)
            loss = criterion(x_hat, x)

            # backprop
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

            train_loss += loss.item()
        
        with torch.no_grad():
            # model.eval()
            test_loss = 0

            for x, label in test_loader:
                model.eval()
                x = x.cuda()

                _, x_hat = model(x)
                loss = criterion(x_hat, x)

                test_loss += loss.item()

        # all_train_loss.append(train_loss/len(train_loader))
        # all_test_loss.append(test_loss/len(test_loader))
        if epoch % 10 == 0:
            print('epoch = {}, train loss = {}, test_loss = {};'.format(epoch,\
                                                train_loss/len(train_loader), test_loss/len(test_loader)))
    return model



# Machine Learning 
def classic_reg(X_train, y_train, X_test, method):
    if method == 'rf':
        rg = RandomForestRegressor()
    if method == 'svr':
        rg = SVR(kernel='rbf')
    if method == 'elasticnet':
        rg = ElasticNet()

    rg.fit(X_train, y_train)
    test_pred = rg.predict(X_test)
    return test_pred

def evaluate(y_test, test_pred):
    r2 = r2_score(y_test, test_pred)
    mse = mean_squared_error(y_test, test_pred)   
    pccs = pearsonr(y_test, test_pred)
    
    # print("Method = {}, r2_score = {}, MSE = {}, PCC = {}, PCC_p".format(method, r2, mse, pccs[0], pccs[1]))
    return r2, mse, pccs[0]

# Deal data
def process_data(drug, data):
    data = data[data.iloc[:,0] == drug]
    labels = np.array(data['Y'])
    new_data = []

    for i in data['X2']:
        for j in i:
            new_data.append(j)

    new_data = np.array(new_data).reshape(-1,17737) 
    return new_data, labels


# Main
def main(data, drug, cut, batch_size):

    print("--------> Processing data <--------")
    data, labels = process_data(drug, data)

    print("--------> Train-Test split <--------")
    # X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)
    num = 1
    all_r2 = [0, 0, 0]
    all_mse = [0, 0, 0]
    all_pcc = [0, 0, 0]
    kf = KFold(n_splits=5,shuffle=True)
    for train_index , test_index in kf.split(data):
        X_train = data[train_index]    
        y_train = labels[train_index]
        X_test = data[test_index]
        y_test = labels[test_index]

        print('***** Fold {} *****'.format(str(num)))
        num += 1

        print("--------> Selecting top variable genes <--------")
        X_train, X_test = top_mad(X_train, X_test, cut)

        print("--------> Min-Max Scaling <--------")
        X_train, X_test = scale_data(X_train, X_test)

        print("--------> Dataloader <--------")
        train_dataset = RegressionDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size)   

        test_dataset = RegressionDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())
        test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)

        print("--------> Training AE <--------")
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = AE_train(X_train.shape[1], train_loader, test_loader)

        train_encoder, _ = model(torch.from_numpy(X_train).float().cuda())
        test_encoder, _ = model(torch.from_numpy(X_test).float().cuda())

        print("--------> machine learning regression <--------")
        X_train = train_encoder.cpu().detach().numpy()
        X_test = test_encoder.cpu().detach().numpy()

        methods = ['rf', 'svr', 'elasticnet']
        for j in range(3):
            test_pred = classic_reg(X_train, y_train, X_test, method=methods[j])
            r2, mse, pcc = evaluate(y_test, test_pred)
            all_r2[j] += r2
            all_mse[j] += mse
            all_pcc[j] += pcc

    with open('gdrive/MyDrive/AE_results/TDC/evaluation_res/drugs_evauation.txt', 'a') as f:
        f.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(drug, all_r2[0]/5, all_mse[0]/5, all_pcc[0]/5, \
                all_r2[1]/5, all_mse[1]/5, all_pcc[1]/5,\
                all_r2[2]/5, all_mse[2]/5, all_pcc[2]/5))
        
    print("Drug: {}; \nRF_R2 = {}; RF_MSE = {}; RF_PCC = {};\n SVM_R2 = {}; SVM_MSE = {}; SVM_PCC = {};\n Elastic_R2 = {}; Elastic_MSE = {}; Elastic_PCC = {}".format(drug, all_r2[0]/5, all_mse[0]/5, all_pcc[0]/5, \
          all_r2[1]/5, all_mse[1]/5, all_pcc[1]/5, \
          all_r2[2]/5, all_mse[2]/5, all_pcc[2]/5))


In [ ]:
pkl_file = open('gdrive/MyDrive/AE_results/TDC/gdsc1.pkl', 'rb')
data = pickle.load(pkl_file)
pkl_file.close()

In [ ]:
for drug in np.unique(data.iloc[:,0]):
    main(data, drug, cut=0.2, batch_size=20)
    torch.cuda.empty_cache()

--------> Processing data <--------
--------> Train-Test split <--------
***** Fold 1 *****
--------> Selecting top variable genes <--------
--------> Min-Max Scaling <--------
--------> Dataloader <--------
--------> Training AE <--------
epoch = 0, train loss = 0.03973236204021507, test_loss = 0.03788176841206021;
epoch = 10, train loss = 0.0247704171989527, test_loss = 0.026705816802051332;
epoch = 20, train loss = 0.02126143707169427, test_loss = 0.023579025227162573;
epoch = 30, train loss = 0.019747818147556648, test_loss = 0.024587115479840174;
epoch = 40, train loss = 0.019241490090886753, test_loss = 0.023986510725484952;
--------> machine learning regression <--------
***** Fold 2 *****
--------> Selecting top variable genes <--------
--------> Min-Max Scaling <--------
--------> Dataloader <--------
--------> Training AE <--------
epoch = 0, train loss = 0.03946060832175943, test_loss = 0.035595263871881694;
epoch = 10, train loss = 0.02489503648959928, test_loss = 0.0239984

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


***** Fold 3 *****
--------> Selecting top variable genes <--------
--------> Min-Max Scaling <--------
--------> Dataloader <--------
--------> Training AE <--------
epoch = 0, train loss = 0.05057761212810874, test_loss = 0.04346957616508007;
epoch = 10, train loss = 0.02985846297815442, test_loss = 0.030818281695246696;
epoch = 20, train loss = 0.026094387518242, test_loss = 0.028344871941953897;
epoch = 30, train loss = 0.02477315382566303, test_loss = 0.027208085171878338;
epoch = 40, train loss = 0.024539471021853387, test_loss = 0.028018931858241558;
--------> machine learning regression <--------
***** Fold 4 *****
--------> Selecting top variable genes <--------
--------> Min-Max Scaling <--------
--------> Dataloader <--------
--------> Training AE <--------
epoch = 0, train loss = 0.048756028059870005, test_loss = 0.041507418267428875;
epoch = 10, train loss = 0.029811603133566678, test_loss = 0.030648392625153065;
epoch = 20, train loss = 0.02583379519637674, test_loss = 0.

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Drug: A-443654; 
RF_R2 = -0.10509844791087149; RF_MSE = 2.144042611378186; RF_PCC = 0.09255376727889991;
 SVM_R2 = -0.08034261280446073; SVM_MSE = 2.0978603513230802; SVM_PCC = 0.09840588322037556;
 Elastic_R2 = -0.014552673122451232; Elastic_MSE = 1.9701766563734666; Elastic_PCC = nan
--------> Processing data <--------
--------> Train-Test split <--------
***** Fold 1 *****
--------> Selecting top variable genes <--------
--------> Min-Max Scaling <--------
--------> Dataloader <--------
--------> Training AE <--------
epoch = 0, train loss = 0.05062753497622907, test_loss = 0.043882839381694794;
epoch = 10, train loss = 0.029919894295744598, test_loss = 0.03246227465569973;
epoch = 20, train loss = 0.026566523243673146, test_loss = 0.031041337177157402;
epoch = 30, train loss = 0.02433348400518298, test_loss = 0.029048395343124866;
epoch = 40, train loss = 0.02274970372673124, test_loss = 0.029810846783220768;
--------> machine learning regression <--------
***** Fold 2 *****
------

...

***** Fold 3 *****
--------> Selecting top variable genes <--------
--------> Min-Max Scaling <--------
--------> Dataloader <--------
--------> Training AE <--------
epoch = 0, train loss = 0.039252568760679826, test_loss = 0.03694387566712168;
epoch = 10, train loss = 0.024335113405767415, test_loss = 0.024738185107707977;
epoch = 20, train loss = 0.021149720220516127, test_loss = 0.02308338073392709;
epoch = 30, train loss = 0.019400709670864873, test_loss = 0.022226311680343416;
epoch = 40, train loss = 0.018958128781782255, test_loss = 0.02362841284937329;
--------> machine learning regression <--------


/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


***** Fold 4 *****
--------> Selecting top variable genes <--------
--------> Min-Max Scaling <--------
--------> Dataloader <--------
--------> Training AE <--------
epoch = 0, train loss = 0.03967779678189092, test_loss = 0.03554353821608755;
epoch = 10, train loss = 0.024660485951850813, test_loss = 0.024034215551283624;
epoch = 20, train loss = 0.021739866636279557, test_loss = 0.022535685863759782;
epoch = 30, train loss = 0.020133282368381817, test_loss = 0.021634864310423534;
epoch = 40, train loss = 0.019222910412483744, test_loss = 0.021877243287033506;
--------> machine learning regression <--------


/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


***** Fold 5 *****
--------> Selecting top variable genes <--------
--------> Min-Max Scaling <--------
--------> Dataloader <--------
--------> Training AE <--------
epoch = 0, train loss = 0.040572212698558964, test_loss = 0.03674432883659998;
epoch = 10, train loss = 0.027938030815372866, test_loss = 0.026380087973343;
epoch = 20, train loss = 0.021943010958946414, test_loss = 0.023172095004055236;
epoch = 30, train loss = 0.021186667546215985, test_loss = 0.022879230479399364;
epoch = 40, train loss = 0.0194110869585226, test_loss = 0.022210867661568854;
--------> machine learning regression <--------
Drug: SGC0946; 
RF_R2 = 0.0820696783626649; RF_MSE = 0.2921225567014495; RF_PCC = 0.32278575879577254;
 SVM_R2 = 0.11356889044926821; SVM_MSE = 0.28204501709530805; SVM_PCC = 0.3535088764798874;
 Elastic_R2 = -0.0008345048759852425; Elastic_MSE = 0.31700281421474985; Elastic_PCC = nan
--------> Processing data <--------
--------> Train-Test split <--------
***** Fold 1 *****
-------->